In [1]:
import pyodbc as odbc
import pandas as pd
from USER import DRIVERS,SERVER,DATABASE
import numpy as np

KHỞI TẠO KẾT NỐI VỚI SQL SERVER

In [ ]:
conn_str = f"""
    DRIVER={DRIVERS};
    SERVER={SERVER};
    DATABASE={DATABASE};
    Trusted_Connection=yes;
"""

conn = odbc.connect(conn_str)

In [3]:
cursor = conn.cursor()

NHẬP DỮ LIỆU VỀ ĐỊA PHƯƠNG VÀO SQL SERVER

In [4]:
data = pd.read_csv('Datasets/Clean_data/Locality_Analyst.csv')

TẠO BẢNG LOCALITY_DATA TRÊN SQL

In [5]:
data.columns

Index(['Vùng', 'Năm', 'Diện tích(Km2)', 'Dân số trung bình (Nghìn người)',
       'Mật độ dân số (Người/km2)', 'Tổng dân số', 'Nam', 'Nữ',
       'Tổng dân số thành thị', 'Tổng dân số nông thôn', 'Tỷ suất sinh thô',
       'Tỷ suất chết thô', 'Tỷ lệ tăng tự nhiên', 'Tỷ suất nhập cư',
       'Tỷ suất xuất cư', 'Tỷ suất di cư thuần', 'Tỷ số giới tính', 'Khu vực',
       'Tỉ lệ thất nghiệp', 'Tỉ lệ thất nghiệp ở thành thị',
       'Tỉ lệ thất nghiệp ở nông thôn', 'Phần trăm dân số'],
      dtype='object')

In [ ]:
cursor = conn.cursor()
cursor.execute("""
    IF OBJECT_ID('dbo.population_data', 'U') IS NOT NULL
        DROP TABLE dbo.population_data;

    CREATE TABLE dbo.population_data (
        region NVARCHAR(100),
        year INT,
        area_km2 FLOAT,
        avg_population_thousand FLOAT,
        population_density FLOAT,
        total_population FLOAT,
        male FLOAT,
        female FLOAT,
        urban_population FLOAT,
        rural_population FLOAT,
        crude_birth_rate FLOAT,
        crude_death_rate FLOAT,
        natural_growth_rate FLOAT,
        immigration_rate FLOAT,
        emigration_rate FLOAT,
        net_migration_rate FLOAT,
        sex_ratio FLOAT,
        area_type NVARCHAR(100),
        unemployment_rate FLOAT,
        urban_unemployment_rate FLOAT,
        rural_unemployment_rate FLOAT,
        population_percentage FLOAT
    );
""")

conn.commit()
cursor.close()


THỰC HIỆN ÉP KIỂU ĐỂ CÓ THỂ ĐƯA LÊN SQL SERVER

In [7]:
data = data.replace('..',np.nan)

col = ['Năm', 'Diện tích(Km2)', 'Dân số trung bình (Nghìn người)',
       'Mật độ dân số (Người/km2)', 'Tổng dân số', 'Nam', 'Nữ',
       'Tổng dân số thành thị', 'Tổng dân số nông thôn', 'Tỷ suất sinh thô',
       'Tỷ suất chết thô', 'Tỷ lệ tăng tự nhiên', 'Tỷ suất nhập cư',
       'Tỷ suất xuất cư', 'Tỷ suất di cư thuần', 'Tỷ số giới tính',
       'Tỉ lệ thất nghiệp', 'Tỉ lệ thất nghiệp ở thành thị',
       'Tỉ lệ thất nghiệp ở nông thôn', 'Phần trăm dân số']

data[col] = data[col].apply(pd.to_numeric,errors='coerce')

data = data.replace(np.nan,None)
data['Vùng'] = data['Vùng'].astype('string')

ĐƯA DỮ LIỆU VỀ ĐỊA PHƯƠNG LÊN SQL SERVER

In [8]:
cursor = conn.cursor()
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO Population_Data (
            region, year, area_km2, avg_population_thousand,
            population_density, total_population, male, female,
            urban_population, rural_population, crude_birth_rate,
            crude_death_rate, natural_growth_rate, immigration_rate,
            emigration_rate, net_migration_rate, sex_ratio, area_type,
            unemployment_rate, urban_unemployment_rate, rural_unemployment_rate,
            population_percentage
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row['Vùng'],
        row['Năm'],
        row['Diện tích(Km2)'],
        row['Dân số trung bình (Nghìn người)'],
        row['Mật độ dân số (Người/km2)'],
        row['Tổng dân số'],
        row['Nam'],
        row['Nữ'],
        row['Tổng dân số thành thị'],
        row['Tổng dân số nông thôn'],
        row['Tỷ suất sinh thô'],
        row['Tỷ suất chết thô'],
        row['Tỷ lệ tăng tự nhiên'],
        row['Tỷ suất nhập cư'],
        row['Tỷ suất xuất cư'],
        row['Tỷ suất di cư thuần'],
        row['Tỷ số giới tính'],
        row['Khu vực'],
        row['Tỉ lệ thất nghiệp'],
        row['Tỉ lệ thất nghiệp ở thành thị'],
        row['Tỉ lệ thất nghiệp ở nông thôn'],
        row['Phần trăm dân số']
    ))


conn.commit()
cursor.close()